In [1]:
import sys
import os
import numpy as np
import pandas as pd
from multiprocessing import Process
sys.path.insert(0, '../../')

In [2]:
from library.notebook_api.data_loader import CombinedDataLoader, ModelDataLoader

<class 'library.notebook_api.data_loader.CombinedDataLoader'>


In [3]:
from library.source_data.feature_extractor import AudioFeatureExtractor

## Load initial Meta

In [4]:
df = CombinedDataLoader().get_combined_df()#.head()


## Feature Extraction on full dataset with fma_small

 Extract just the features into memory and then persist to parquet

In [62]:
#instantiate the extractor with all data with non null label

#This takes about 90 minutes to actually do full one 

#To do modify to smaller subset
# AND create some batch processor to chunk out the steps to fit in memory
#also - import and review bowie dataset as alternate


#full_extract = AudioFeatureExtractor(df[df.label.isnull() ==False])
#full_extract = AudioFeatureExtractor(df[df.label.isnull() ==False].sample(1000))

full_extract = AudioFeatureExtractor(df[df.label.isnull() ==False].sample(500))






In [10]:
batch_input = df[df.label.isnull() ==False].sample(2400).copy()

In [16]:
class AudioParallelProcessor():
    def __init__(self, source_data, version = '003', batch=1, threads = 5):
        self.version = version 
        self.batch = batch
        self.threads = threads
        self.source_data = source_data
        self.input_length = len(self.source_data)
        self.start_thread_size = int(self.input_length/self.threads)

        return
    
    def execute(self):
        self.run_processes(self.get_extract_processes())
    
    def get_extract_processes(self):
        
        #map of index number and the function to run 
        extract_thread_functions = {}
        #map of index number and process to execture 
        extract_processes = {}
        #1 based indexes for the threads 
        thread_indexes = list(range(1,self.threads+1))
   
        #Instantiate the index locations for getting rows from data frame 
        start_record = 0
        end_record = self.start_thread_size 

        #for reach thread build out the processes for a subset of the data 
        for thread in thread_indexes:
            if thread == 2:
                start_record +=1
            if end_record == self.input_length - self.input_length%self.threads:
                end_record += self.input_length%self.threads
            print(start_record, end_record, len(batch_input.iloc[start_record:end_record]))
            start_record += self.start_thread_size
            end_record += self.start_thread_size 
            #extract_thread_functions[thread] = AudioFeatureExtractor(df[df.label.isnull() ==False].sample(100))
            extract_thread_functions[thread] = AudioFeatureExtractor(self.source_data.iloc[start_record:end_record])
            extract_processes[thread] = Process(target =extract_thread_functions[thread].run_extraction_thread,args=(self.version,self.batch,thread))
        return extract_processes


    def run_processes(self,extract_processes):
        for thread_index, process in extract_processes.items():
            process.start()
        for thread_index, process in extract_processes.items():
            process.join()



In [17]:
batch = AudioParallelProcessor(batch_input,version = '004', batch=1, threads = 12)

In [18]:
batch.execute()

0 200 200
201 400 199
401 600 199
601 800 199
801 1000 199
1001 1200 199
1201 1400 199
1401 1600 199
1601 1800 199
1801 2000 199
2001 2200 199
2201 2400 199


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:39: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(PROJECT_ABSOLUTE_PATH+file_name, sr=None)
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:39: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(PROJECT_ABSOLUTE_PATH+file_name, sr=None)
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_cap

In [13]:
batch.execute()

0 400 400
401 800 399
801 1200 399
1201 1600 399
1601 2000 399


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:39: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(PROJECT_ABSOLUTE_PATH+file_name, sr=None)
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:39: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(PROJECT_ABSOLUTE_PATH+file_name, sr=None)
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:39: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(PROJECT_ABSOLUTE_PATH+file_name, sr=None)
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/audio.py:18

In [63]:
#run the extraction to put results to memory
full_extract.add_audio_data_to_df()

Processing File  /project_data_source/free_music_archive/fma_small/121/121866.mp3
Run Librosa Load
extracting_features


/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:118: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)[0]


Processing File  /project_data_source/free_music_archive/fma_small/043/043637.mp3
Run Librosa Load
Processing File  /project_data_source/free_music_archive/fma_small/056/056173.mp3
Run Librosa Load
Processing File  /project_data_source/free_music_archive/fma_small/079/079449.mp3
Run Librosa Load
Processing File  /project_data_source/free_music_archive/fma_small/095/095382.mp3
Run Librosa Load
Processing File  /project_data_source/free_music_archive/fma_small/094/094288.mp3
Run Librosa Load
Processing File  /project_data_source/free_music_archive/fma_small/106/106108.mp3
Run Librosa Load
Processing File  /project_data_source/free_music_archive/fma_small/064/064775.mp3
Run Librosa Load
Processing File  /project_data_source/free_music_archive/fma_small/137/137113.mp3
Run Librosa Load
Processing File  /project_data_source/free_music_archive/fma_small/141/141993.mp3
Run Librosa Load
Processing File  /project_data_source/free_music_archive/fma_small/087/087303.mp3
Run Librosa Load
Processing

/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:39: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(PROJECT_ABSOLUTE_PATH+file_name, sr=None)
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/.venv/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Processing File  /project_data_source/free_music_archive/fma_small/089/089447.mp3
Run Librosa Load
extracting_features
Processing File  /project_data_source/free_music_archive/fma_small/127/127187.mp3
Run Librosa Load
extracting_features
Processing File  /project_data_source/free_music_archive/fma_small/043/043619.mp3
Run Librosa Load
Processing File  /project_data_source/free_music_archive/fma_small/100/100930.mp3
Run Librosa Load
Processing File  /project_data_source/free_music_archive/fma_small/043/043895.mp3
Run Librosa Load
Processing File  /project_data_source/free_music_archive/fma_small/140/140150.mp3
Run Librosa Load
Processing File  /project_data_source/free_music_archive/fma_small/086/086144.mp3
Run Librosa Load
Processing File  /project_data_source/free_music_archive/fma_small/113/113995.mp3
Run Librosa Load
Processing File  /project_data_source/free_music_archive/fma_small/136/136710.mp3
Run Librosa Load
Processing File  /project_data_source/free_music_archive/fma_small/10

In [7]:
#save the results to disk in version 001 file
full_extract.save_results('002') 

/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_save['fma_genres'] = to_save['fma_genres'].astype('string')
/Users/murphybre/Desktop/UMichigan/Fall2024/siads699_capstone/music_modeling_capstone/notebooks/data_creation/../../library/source_data/feature_extractor.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_save['fma_genres_all'] = to_save

In [8]:
full_model_data = ModelDataLoader('001')

In [9]:
full_model_data.df.count()

track_id                              8996
dataset                               8996
audio_path                            8996
label                                 8996
fma_genre_top                         8996
fma_genres                            8996
fma_genres_all                        8996
sampling_rate                         8996
features                              8996
spectral_centroids_mean               8996
spectral_centroids_delta_mean         8996
spectral_centroids_accelerate_mean    8996
spectral_bandwidth_mean               8996
spectral_rolloff_mean                 8996
zero_crossing_rate_mean               8996
rms_mean                              8996
chroma_stft_mean                      8996
mfccs_mean                            8996
dtype: int64

In [ ]:
#To do/not finished: start of batch processer which can aid in running extractor against many smaller chunks of data
#resulting in multiple output files saved

class AudioBatchProcesser():
    def __init__(self, source_data, batch_size):
        self.df = source_data
        self.batch_size = batch_size 

    self run_feature_extraction(self):
        batch_start = 0 
        batch_count = len(self.df) / self. batch_size 


## Extraction while saving audio and features on small sample
in this run, we saved the output of librosa load in the parquet as well.  This takes a lot of memory so only did a small example

In [4]:
#instantiate data 
#
gtza_test_data = df[df.dataset == 'gtzan']

In [5]:
#instantiate extractor
#commented in alternate return classe in get_audio_data method to include full audio
gtza_extract = AudioFeatureExtractor(gtza_test_data.head(5))

In [6]:
#run extraction
#print statement uncommented 
gtza_extract.add_audio_data_to_df()

Processing File  /project_data_source/gtzan_dataset/Data/genres_original/blues/blues.00000.wav
Run Librosa Load
extracting_features
Processing File  /project_data_source/gtzan_dataset/Data/genres_original/blues/blues.00001.wav
Run Librosa Load
extracting_features
Processing File  /project_data_source/gtzan_dataset/Data/genres_original/blues/blues.00002.wav
Run Librosa Load
extracting_features
Processing File  /project_data_source/gtzan_dataset/Data/genres_original/blues/blues.00003.wav
Run Librosa Load
extracting_features
Processing File  /project_data_source/gtzan_dataset/Data/genres_original/blues/blues.00004.wav
Run Librosa Load
extracting_features
putting features to their own columns


In [7]:
#save the results
gtza_extract.save_results() 

In [10]:
#Load the model data for this version
model_data = ModelDataLoader('000')

In [11]:
#Note full librosa_load output array was also saved in this parquet
model_data.df

,dataset,audio_path,label,fma_genre_top,fma_genres,fma_genres_all,librosa_load,sampling_rate,features,spectral_centroids_mean,spectral_centroids_delta_mean,spectral_centroids_accelerate_mean,spectral_bandwidth_mean,spectral_rolloff_mean,zero_crossing_rate_mean,rms_mean,chroma_stft_mean,mfccs_mean
track_id,,,,,,,,,,,,,,,,,,
blues.00000.wav,gtzan,/project_data_source/gtzan_dataset/Data/genres...,blues,n/a,n/a,n/a,"[0.0073242188, 0.016601562, 0.0076293945, -0.0...",22050,"[1784.1227, -0.49052292, -0.061881434, 2002.41...",1784.122681,-0.490523,-0.061881,2002.412354,3805.723145,0.083045,0.130184,0.350129,3.121230
blues.00001.wav,gtzan,/project_data_source/gtzan_dataset/Data/genres...,blues,n/a,n/a,n/a,"[0.0034179688, 0.0043029785, 0.001373291, 0.00...",22050,"[1530.2617, 0.6459638, -0.0855879, 2038.9875, ...",1530.261719,0.645964,-0.085588,2038.987549,3550.713623,0.056040,0.095908,0.340849,-0.007777
blues.00002.wav,gtzan,/project_data_source/gtzan_dataset/Data/genres...,blues,n/a,n/a,n/a,"[0.019012451, 0.047698975, 0.029418945, -0.018...",22050,"[1552.8325, 0.3400274, 0.025039379, 1747.754, ...",1552.832520,0.340027,0.025039,1747.754028,3042.410156,0.076291,0.175473,0.363538,3.364070
blues.00003.wav,gtzan,/project_data_source/gtzan_dataset/Data/genres...,blues,n/a,n/a,n/a,"[-0.013000488, -0.03060913, -0.036071777, -0.0...",22050,"[1070.1534, 0.25125474, 0.07936938, 1596.4226,...",1070.153442,0.251255,0.079369,1596.422607,2184.879150,0.033309,0.141040,0.404854,0.866870
blues.00004.wav,gtzan,/project_data_source/gtzan_dataset/Data/genres...,blues,n/a,n/a,n/a,"[-0.0063171387, -0.009277344, -0.008331299, -0...",22050,"[1835.1285, -0.17207043, -0.24526799, 1748.410...",1835.128540,-0.172070,-0.245268,1748.410767,3579.957520,0.101461,0.091501,0.308526,-9.653034
